In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import xml.etree.ElementTree as xet
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
df = pd.read_csv('./labels_2.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images/Cars213.xml,150,233,160,194
1,./images/Cars207.xml,254,372,229,264
2,./images/Cars398.xml,72,196,139,222
3,./images/Cars367.xml,224,382,161,222
4,./images/N107.xml,207,356,174,287


In [3]:
filename = df['filepath'][0]

In [4]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image= os.path.join('.\images', filename_image)
    return filepath_image

In [5]:
getFilename(filename)

'.\\images/Cars213.png'

In [6]:
image_path = list(df['filepath'].apply(getFilename))
print(len(image_path))

658


In [7]:
# Verify image output
file_path = image_path[0]
img = cv2.imread(file_path)

# cv2.namedWindow('example', cv2.WINDOW_NORMAL)
# cv2.imshow('example', img)
# cv2.waitKey(1000)
# cv2.destroyAllWindows()

[ WARN:0@24.364] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('.\images/Cars213.png'): can't open/read file: check file path/integrity


In [28]:
# 207	356	174	287 coordinates for img
cv2.rectangle(img, (207, 174), (356, 287), (0, 255, 0), 3)
cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example', img)
cv2.waitKey(1000)
cv2.destroyAllWindows()

: 

Data Preprocessing

In [8]:
labels = df.iloc[:, 1:].values

In [9]:
data = []
output = []

for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h, w, d = img_arr.shape

    #preprocessing
    load_image = load_img(image, target_size=(224, 224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0

    #normalization to labels
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    label_norm = (nxmin, nxmax, nymin, nymax)

    # append values
    data.append(norm_load_image_arr)
    output.append(label_norm)

[ WARN:0@29.287] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('.\images/Cars213.png'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [10]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [11]:
print(X.shape, y.shape)

(225, 224, 224, 3) (225, 4)


In [12]:
# dividing the data to train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

### Object Detection Model


In [13]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [14]:
# building the network
inception_resnet = InceptionResNetV2(weights="imagenet", include_top=False, 
                                    input_tensor=Input(shape=(224, 224, 3)))
inception_resnet.trainable = False
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation="relu")(headmodel)
headmodel = Dense(250, activation="relu")(headmodel)
headmodel = Dense(4, activation="sigmoid")(headmodel)

# model
model = Model(inputs=inception_resnet.input, outputs=headmodel)

2022-07-17 16:32:29.593071: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-17 16:32:29.593196: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [16]:
# compile model
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [17]:
from tensorflow.keras.callbacks import TensorBoard

In [18]:
tfb = TensorBoard('object_detection')

In [19]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=100,    
                    validation_data=(x_test, y_test), callbacks=[tfb])

Epoch 1/100


2022-07-17 16:38:17.070313: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-17 16:38:22.117547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - ETA: 0s - loss: 0.1467

2022-07-17 16:38:33.740478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 21s 552ms/step - loss: 0.1467 - val_loss: 0.1348
Epoch 2/100
18/18 [==============================] - 4s 223ms/step - loss: 0.1193 - val_loss: 0.0848
Epoch 3/100
18/18 [==============================] - 4s 221ms/step - loss: 0.1057 - val_loss: 0.0918
Epoch 4/100
18/18 [==============================] - 4s 241ms/step - loss: 0.0964 - val_loss: 0.0877
Epoch 5/100
18/18 [==============================] - 4s 229ms/step - loss: 0.0961 - val_loss: 0.0996
Epoch 6/100
18/18 [==============================] - 4s 225ms/step - loss: 0.0960 - val_loss: 0.0872
Epoch 7/100
18/18 [==============================] - 4s 225ms/step - loss: 0.0934 - val_loss: 0.0940
Epoch 8/100
18/18 [==============================] - 4s 223ms/step - loss: 0.1012 - val_loss: 0.1097
Epoch 9/100
18/18 [==============================] - 4s 217ms/step - loss: 0.0974 - val_loss: 0.0959
Epoch 10/100
18/18 [==============================] - 4s 220ms/step - loss: 0.0911 - val_loss: 0.0883


In [20]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=200,    
                    validation_data=(x_test, y_test), callbacks=[tfb], initial_epoch=100)

Epoch 101/200
18/18 [==============================] - 6s 253ms/step - loss: 4.5519e-04 - val_loss: 0.0100
Epoch 102/200
18/18 [==============================] - 4s 224ms/step - loss: 3.3376e-04 - val_loss: 0.0097
Epoch 103/200
18/18 [==============================] - 4s 234ms/step - loss: 3.0894e-04 - val_loss: 0.0093
Epoch 104/200
18/18 [==============================] - 4s 234ms/step - loss: 3.5391e-04 - val_loss: 0.0095
Epoch 105/200
18/18 [==============================] - 4s 231ms/step - loss: 4.1098e-04 - val_loss: 0.0093
Epoch 106/200
18/18 [==============================] - 4s 231ms/step - loss: 3.5084e-04 - val_loss: 0.0097
Epoch 107/200
18/18 [==============================] - 4s 225ms/step - loss: 3.4839e-04 - val_loss: 0.0096
Epoch 108/200
18/18 [==============================] - 4s 233ms/step - loss: 2.9044e-04 - val_loss: 0.0095
Epoch 109/200
18/18 [==============================] - 4s 227ms/step - loss: 2.6660e-04 - val_loss: 0.0096
Epoch 110/200
18/18 [================

In [21]:
model.save("./models/trained_object_detection.h5")